In [7]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=4000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f74a39e6390>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_convnet_model', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
I

INFO:tensorflow:loss = 0.5946294, step = 3099
INFO:tensorflow:probabilities = [[0.00003308 0.00002849 0.00420707 0.00005578 0.02576956 0.00006423
  0.9692336  0.00000005 0.0003134  0.00029476]
 [0.00142908 0.00000156 0.04721313 0.00000413 0.00354293 0.00005097
  0.94769025 0.00000052 0.0000365  0.00003105]
 [0.00040213 0.02507923 0.80103457 0.00177926 0.0177579  0.00039835
  0.09980761 0.00006585 0.0504507  0.00322435]
 [0.00034609 0.00004916 0.00001484 0.00580877 0.00045724 0.00108578
  0.00000321 0.87212497 0.0012043  0.1189056 ]
 [0.00193394 0.00116985 0.00027018 0.00340951 0.0005259  0.85544175
  0.00006442 0.00047494 0.13589175 0.00081765]
 [0.01778409 0.00109805 0.04003143 0.00987951 0.6285947  0.04468786
  0.1219715  0.00186682 0.00245996 0.13162613]
 [0.00368495 0.02545134 0.01386158 0.45011955 0.01518379 0.1734646
  0.00482942 0.11641917 0.05403369 0.142952  ]
 [0.00123506 0.         0.00008179 0.9938631  0.00000001 0.00167363
  0.00000127 0.00054615 0.00259406 0.00000489]
 [0

INFO:tensorflow:global_step/sec: 3.36655
INFO:tensorflow:probabilities = [[0.00008733 0.90251315 0.00466964 0.03386608 0.00128957 0.00380722
  0.02117503 0.0017957  0.02764894 0.0031474 ]
 [0.00036128 0.98113716 0.0031301  0.00161269 0.00270384 0.00071394
  0.00321971 0.00191074 0.00469541 0.00051515]
 [0.04360538 0.06009773 0.03396107 0.01785751 0.00242584 0.0090105
  0.01617081 0.00118649 0.79880357 0.01688105]
 [0.0000383  0.00021557 0.00002542 0.00132171 0.01337253 0.00069083
  0.00001863 0.0713269  0.00913376 0.9038563 ]
 [0.00000051 0.00000117 0.00000081 0.00024217 0.00000283 0.00000215
  0.00000001 0.9995389  0.00003299 0.00017855]
 [0.00024188 0.01579515 0.00011795 0.00248288 0.6458078  0.0192909
  0.00374236 0.05831966 0.16497257 0.08922879]
 [0.00065149 0.00760815 0.00032459 0.00331766 0.55081135 0.02459724
  0.00156956 0.0129265  0.06503979 0.33315367]
 [0.00006367 0.00043799 0.9781332  0.01454084 0.0000314  0.00003466
  0.0001646  0.00315482 0.00332739 0.00011145]
 [0.00067

INFO:tensorflow:loss = 0.39767665, step = 3199 (29.708 sec)
INFO:tensorflow:probabilities = [[0.00135058 0.01073503 0.01462789 0.0006847  0.00397692 0.03309419
  0.9237962  0.00003418 0.01098929 0.00071097]
 [0.00300651 0.00000128 0.24908245 0.00023548 0.0117518  0.00049359
  0.6574479  0.00014199 0.07740795 0.00043107]
 [0.00004519 0.00000381 0.00014611 0.9961522  0.00000104 0.00286851
  0.000001   0.00011819 0.00062036 0.00004354]
 [0.00736488 0.0000007  0.00009938 0.15176009 0.00005001 0.82890165
  0.00032465 0.00807308 0.002675   0.0007506 ]
 [0.01723872 0.15669063 0.00434855 0.04623385 0.00058325 0.34900215
  0.00057658 0.02711845 0.3904131  0.00779466]
 [0.00057517 0.01782905 0.02412253 0.83366126 0.0097821  0.00961697
  0.00274522 0.00984053 0.0265332  0.065294  ]
 [0.00009623 0.000034   0.02535459 0.00012107 0.00204934 0.0015415
  0.9510125  0.00000272 0.0176862  0.00210188]
 [0.00010515 0.00613759 0.00019661 0.00050406 0.7989047  0.00799713
  0.00118394 0.00082684 0.01209845 0

INFO:tensorflow:global_step/sec: 3.37685
INFO:tensorflow:probabilities = [[0.00004978 0.00005594 0.00017205 0.00044303 0.884811   0.00037926
  0.00043578 0.00096983 0.00309967 0.10958364]
 [0.76568615 0.00006409 0.00989573 0.01947205 0.00051981 0.05765851
  0.10814445 0.0002273  0.03552543 0.00280636]
 [0.00008557 0.00000198 0.00001489 0.00042563 0.00015982 0.00707456
  0.00012017 0.00090612 0.94802225 0.04318906]
 [0.0001408  0.00000105 0.9979267  0.00033684 0.00002829 0.00004838
  0.00148858 0.0000001  0.00002825 0.00000099]
 [0.9981053  0.00000013 0.00013099 0.00078395 0.00000149 0.00084785
  0.00004033 0.00000839 0.00006136 0.00002016]
 [0.00714108 0.02144153 0.02812608 0.18376882 0.0078974  0.15251695
  0.04314534 0.00407039 0.47560322 0.07628919]
 [0.9212502  0.00000438 0.00138714 0.00128596 0.00013279 0.03113426
  0.00131621 0.04058001 0.00118367 0.00172542]
 [0.02702073 0.00034361 0.10670315 0.00104756 0.00030098 0.13131714
  0.7298554  0.00000392 0.00334213 0.00006526]
 [0.003

...

INFO:tensorflow:loss = 0.45580268, step = 6899 (28.072 sec)
INFO:tensorflow:probabilities = [[0.00200057 0.00045907 0.34638765 0.02735795 0.00000259 0.00291784
  0.00001375 0.00037788 0.6193783  0.00110438]
 [0.00000411 0.00000894 0.00005308 0.9985373  0.00001128 0.00098255
  0.00000087 0.00000621 0.0003864  0.00000942]
 [0.0001101  0.00149831 0.00030345 0.00099255 0.00001128 0.00143477
  0.00000853 0.0051878  0.97290987 0.01754337]
 [0.00010756 0.00000124 0.99734914 0.00245381 0.00000152 0.0000067
  0.00000223 0.00000501 0.00001482 0.00005788]
 [0.00026801 0.04053232 0.00725898 0.8831832  0.00148231 0.01503952
  0.00043782 0.00969639 0.02004267 0.02205878]
 [0.913367   0.         0.00001134 0.00003141 0.00000029 0.0710033
  0.00002471 0.0118751  0.00078109 0.00290574]
 [0.0005116  0.00000283 0.00146961 0.01469555 0.00000147 0.97101194
  0.0000071  0.0000008  0.01194571 0.00035346]
 [0.0000017  0.00001404 0.00247655 0.9753533  0.0000004  0.00001486
  0.00000086 0.00003181 0.02208128 0.

INFO:tensorflow:global_step/sec: 3.47976
INFO:tensorflow:probabilities = [[0.00081585 0.00000172 0.00000928 0.00000207 0.00000593 0.96803343
  0.00015316 0.00000516 0.03089337 0.00007995]
 [0.00014647 0.00000684 0.00010133 0.00049858 0.00000643 0.00005972
  0.00000002 0.98583996 0.00093937 0.01240145]
 [0.99946815 0.         0.00000599 0.00021803 0.         0.00029874
  0.00000418 0.00000366 0.00000087 0.0000005 ]
 [0.00014091 0.00000157 0.00000471 0.00001359 0.00015231 0.99886876
  0.00009473 0.00000219 0.00056144 0.00015967]
 [0.00298938 0.00958233 0.638909   0.2770557  0.02543508 0.00030272
  0.03771344 0.00027539 0.00546552 0.00227146]
 [0.00024856 0.0032875  0.00011404 0.00003567 0.00089726 0.98375815
  0.00080177 0.00004209 0.01072053 0.00009442]
 [0.00002337 0.00006372 0.00003307 0.00032941 0.02349459 0.00380765
  0.0000141  0.00973686 0.01176256 0.9507346 ]
 [0.00000021 0.00000003 0.9999294  0.00000183 0.00000007 0.00000022
  0.00006729 0.         0.00000108 0.00000003]
 [0.000

INFO:tensorflow:loss = 0.3059155, step = 6999 (28.742 sec)
INFO:tensorflow:probabilities = [[0.0000145  0.00001085 0.00005099 0.0016601  0.00000171 0.00084659
  0.00000271 0.00000306 0.99727803 0.0001315 ]
 [0.00027241 0.9818818  0.00398139 0.00085634 0.00082606 0.00018467
  0.00200985 0.00461378 0.00499181 0.00038198]
 [0.00020038 0.00000122 0.00418792 0.00000034 0.0003199  0.00001068
  0.99527884 0.00000001 0.0000004  0.00000045]
 [0.16318108 0.00017185 0.03341782 0.0131912  0.01600185 0.05299709
  0.70883834 0.00187649 0.0046996  0.00562463]
 [0.00272721 0.00004242 0.00182482 0.03819944 0.9156851  0.02286816
  0.01340387 0.00007974 0.00432023 0.00084903]
 [0.00003955 0.00014878 0.01502436 0.982326   0.0000006  0.00054639
  0.00000057 0.00002537 0.00169837 0.00018999]
 [0.02899249 0.00003077 0.11900894 0.84492934 0.00000287 0.00329049
  0.00005628 0.00021292 0.00314933 0.00032649]
 [0.00011644 0.00014048 0.99853444 0.00071542 0.00001128 0.00003063
  0.00031222 0.00000046 0.00013326 0

INFO:tensorflow:Saving checkpoints for 7098 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.22228263.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-27-18:08:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-7098
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-27-18:09:00
INFO:tensorflow:Saving dict for global step 7098: accuracy = 0.9394, global_step = 7098, loss = 0.21065915
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7098: /tmp/mnist_convnet_model/model.ckpt-7098
{'loss': 0.21065915, 'global_step': 7098, 'accuracy': 0.9394}


SystemExit: 